In [ ]:
## Проработка исходника Джанго,а также фрагменты документации

"""
Цель: научиться писать профессионально, методом переваривания понятного и непонятного

"""

In [32]:
class ContextMixin:
    """
    Миксин контекста по умолчанию, который передает аргументы ключевого слова, полученные
     get_context_data() в качестве контекста шаблона.
     
     "None" часто используется как значение по умолчанию для переменных, которым еще не было присвоено значение, или как контрольное значение, 
     указывающее на конец списка или отсутствие элемента в словаре.
     
     # https://github.com/django/django/blob/40217d1a82b0c16cddba377325d12b2c253f402a/django/views/generic/base.py#L21
    
    
    Двойные звездочки ( **) перед именем параметра kwargs указывают на то, что параметр является словарем
    
    """

    extra_context = None          #  дополнительный атрибут с значением "ничего"
    

    def get_context_data(self, **kwargs):    # цель функции - обработать ключевые аргументы и 
        kwargs.setdefault("view", self)      # метод словаря устанавливает значение по умолчанию "view", если ключ не существует
        if self.extra_context is not None:
            kwargs.update(self.extra_context)
        return kwargs                        # возвращает значения ключа
    
    #get_context_data()
    #get_context_data(None)

     

In [30]:
"""
Возвращает словарь, представляющий контекст шаблона.
Предоставленные аргументы ключевого слова будут составлять возвращаемый контекст. Пример использования:

# https://docs.djangoproject.com/en/4.1/ref/class-based-views/mixins-simple/
"""

def get_context_data(self, **kwargs):
    context = super().get_context_data(**kwargs)      # позволяет ссылаться на род. класс .обьекта 
    context['number'] = random.randrange(1, 100)      # тут идёт добавление в словарь значения
    return context                                    # квадратные скобки указывают на доступ к значению
    print(context)
get_context_data("view")           # супер не может найти род. класс , так как это всего лишь фрагмент

# на сколько я понимаю класс находится выше, но пока что еще картина маслом
# начал прорабатывать родительский класс base.py, чтобы понять что же использует класс наследник

RuntimeError: super(): __class__ cell not found

In [31]:
# подходим к классу наследнику в list.py
# https://github.com/django/django/blob/main/django/views/generic/list.py

In [33]:
from django.db.models import QuerySet
from django.core.paginator import Paginator

class MultipleObjectMixin(ContextMixin):          # является подклассом, класса выше
    """Предоставляет ряд атрибутов и методов, которые можно использовать для настройки поведения универсальных представлений"""
    # констант (True, None) находится во встроенном пространстве имен
    allow_empty = True                       # атрибут для разрешения пустых наборов запросов
    queryset = None
    model = None
    paginate_by = None                       # атрибут который разбивает на страницы
    paginate_orphans = 0
    context_object_name = None
    paginator_class = Paginator                   
    page_kwarg = "page"
    ordering = None
    
    # это всё подкапотная часть, получается следуя документации мы вытягиваем те артибуеты которые нам нужны и используем на практике
    def get_queryset(self):                 # метод который используется для извлечения объектов из БД
        """
        Вернуть список элементов для этого представления.
        Возвращаемое значение должно быть итерируемым и может быть экземпляром
        `QuerySet`, и в этом случае будет включено конкретное поведение `QuerySet`.
        
        """
        if self.queryset is not None:            # если артибут не установлен, то будет использоваться по умолчанию
            queryset = self.queryset             # вывести обьекты 
            if isinstance(queryset, QuerySet):   # isinstance пробивает информацию, да или нет
                queryset = queryset.all()        # выводит списко обьектов
        elif self.model is not None:             # если такой модели вообще не существует, то 
            queryset = self.model._default_manager.all()             
        else:
            raise ImproperlyConfigured(          # иначе исключение и предлагает сделать следующие действия
                "%(cls)s is missing a QuerySet. Define "
                "%(cls)s.model, %(cls)s.queryset, or override "
                "%(cls)s.get_queryset()." % {"cls": self.__class__.__name__}
            )
        ordering = self.get_ordering()       # цель: получить информацию о порядке 
        if ordering:             
            if isinstance(ordering, str):    # если ordering это строка, то отсортируй
                ordering = (ordering,)
            queryset = queryset.order_by(*ordering)

        return queryset
    

    def get_ordering(self):          # эта функция на сколько я понимаю была создана для того, чтобы сделать пагинацию выше
        """ функция возвращает значение ordering атрибута """
        return self.ordering
    

    def paginate_queryset(self, queryset, page_size):
        """При необходимости разбейте набор запросов на страницы"""
        paginator = self.get_paginator(
            queryset,
            page_size,
            orphans=self.get_paginate_orphans(),
            allow_empty_first_page=self.get_allow_empty(),
        )
        page_kwarg = self.page_kwarg
        page = self.kwargs.get(page_kwarg) or self.request.GET.get(page_kwarg) or 1
        try:
            page_number = int(page)
        except ValueError:
            if page == "last":
                page_number = paginator.num_pages
            else:
                raise Http404(
                    _("Page is not “last”, nor can it be converted to an int.")
                )
        try:
            page = paginator.page(page_number)
            return (paginator, page, page.object_list, page.has_other_pages())
        except InvalidPage as e:
            raise Http404(
                _("Invalid page (%(page_number)s): %(message)s")
                % {"page_number": page_number, "message": str(e)}
            )
# отдельные методы можно назвать слоями, теми кусками которые полноценно используются для отображения списка обьектов
        

ModuleNotFoundError: No module named 'django'